In [24]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver import FirefoxOptions
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd

from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver.firefox.service import Service

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from selenium.common.exceptions import NoSuchElementException
import pickle

import re

In [2]:
opts = FirefoxOptions()
opts.add_argument("--width=800")
opts.add_argument("--height=600")
#opts.add_argument('--headless')
service = Service("/snap/bin/firefox.geckodriver")

In [3]:
navegador = webdriver.Firefox(options = opts, service=service)

In [4]:
script = """
navigator.geolocation.getCurrentPosition = function(success){
    var position = {"coords" : {"latitude": "34.0207305","longitude": "-118.6919153"}};
    success(position);
}
"""
navegador.execute_script(script)

In [5]:
navegador.get('https://www.amazon.com/dp/B006UACRTG/')

In [6]:
delta_y = 10000
navegador.execute_script(f"window.scrollBy(0, {delta_y})")

In [8]:
'''
# LOGIC FOR REVIEWS AND SORT DROPDOWN
reviews = navegador.find_element(By.ID, 'customerReviews')

dropdown_list = navegador.find_elements(By.XPATH, '//span[@data-csa-c-func-deps="aui-da-a-dropdown-button"]')
dropdown_list[1].text
'''

'Top reviews'

In [9]:
see_all_reviews = navegador.find_element(By.XPATH, '//a[@data-hook="see-all-reviews-link-foot"]')

# Rola a página até o elemento
navegador.execute_script("arguments[0].scrollIntoView();", see_all_reviews)
navegador.execute_script("window.scrollBy(0, -150)")

see_all_reviews.click()

In [16]:
### PAGINA DE REVIEWS

In [11]:
#stars = navegador.find_elements(By.XPATH, '//tr[starts-with(@aria-label, "5 stars")]')
stars = {}
for i in range(1, 6):
    star = navegador.find_element(By.XPATH, f'//tr[starts-with(@aria-label, "{i} stars")]')
    stars[i] = star.text.split('\n')[1]
stars

{1: '32%', 2: '11%', 3: '8%', 4: '38%', 5: '12%'}

In [12]:
dropdown_filters = navegador.find_elements(By.XPATH, '//span[@data-csa-c-func-deps="aui-da-a-dropdown-button"]')

for sort in dropdown_filters:
    print(sort.text)

sort_by = dropdown_filters[0]
star_filter = dropdown_filters[2]

Top reviews
All reviewers
All stars
Text, image, video


In [13]:
reviews_location = navegador.find_element(By.XPATH, '//h3[@data-hook="arp-local-reviews-header"]')
reviews_location.text

'From the United States'

In [14]:
reviews = navegador.find_elements(By.XPATH, '//div[@id="cm_cr-review_list"]//div[starts-with(@id, "customer_review-")]')
reviews[0].text

'Afterhourshop\nNot bad AT ALL. Better for me than the $2K X-Rite software I spent.\nReviewed in the United States on October 26, 2020\nVerified Purchase\nOk, so I have the $2K i1 Pro2 and I got this as I wanted to compare them.\nI am NOT going into deep details, but I can say that this hands down does a great job of making your prints MUCH better with fine tuning the colors and tones. I used the Expert patchs of about 750 and with Gray tones, and I used the Click method of EACH patch of color. This took some time and patients, BUT, it is the right way to go. The other method is LAME, and dont EVEN think about printing on a large sheet with more patches. WASTE of time. Print on a 8.5x11 or something so that cheap plastic paper guide that is included can actually help. This is where the i1 X-Rite is far superior, but SURELY not worth the $ extra for personal use.\n\nSo do the readings patch at a time, and you will be sure you got a good reading. My BW prints using color inks is not nutr

In [27]:
# encontra sequências de números seguidos por um espaço
pattern = r'\d+(?=\s)'

review_count = navegador.find_element(By.XPATH, '//div[@data-hook="cr-filter-info-review-rating-count"]')
review_count_text = review_count.text

# número total de reviews
int(re.findall(pattern, review_count_text)[1])

28

In [15]:
'''
# LÓGICA UTILIZADA PARA NEXT PAGE


try:
    next_page = navegador.find_element(By.XPATH, '//div[@id="cm_cr-pagination_bar"]//li[@class="a-last"]')
    next_page.click()
except:
    print('Não foi possível encontrar mais avaliações.')
    pass
'''

'\n# LÓGICA UTILIZADA PARA NEXT PAGE\n\n\ntry:\n    next_page = navegador.find_element(By.XPATH, \'//div[@id="cm_cr-pagination_bar"]//li[@class="a-last"]\')\n    next_page.click()\nexcept:\n    print(\'Não foi possível encontrar mais avaliações.\')\n    pass\n'

In [35]:
all_reviews_text = []
all_stars_text = []
delta_y = 10000

while True:
    try:
        navegador.execute_script(f"window.scrollBy(0, {delta_y})")
        WebDriverWait(navegador, 3).until(EC.invisibility_of_element_located((By.XPATH, '//div[@class="a-section cr-list-loading reviews-loading"]')))
        
        reviews = navegador.find_elements(By.XPATH, '//div[@id="cm_cr-review_list"]//div[starts-with(@id, "customer_review-")]')
        stars = navegador.find_elements(By.XPATH, '//i[@data-hook="review-star-rating"]')
        stars = [star.get_attribute('class') for star in stars]

        all_reviews_text.append([review.text for review in reviews])
        all_stars_text.append(stars)

        next_page = navegador.find_element(By.XPATH, '//div[@id="cm_cr-pagination_bar"]//li[@class="a-last"]')

        navegador.execute_script("arguments[0].scrollIntoView();", next_page)
        navegador.execute_script("window.scrollBy(0, -150)")

        next_page = WebDriverWait(navegador, 3).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="cm_cr-pagination_bar"]//li[@class="a-last"]')))
        
        
        #retrying_find_click(By.XPATH, '//div[@id="cm_cr-pagination_bar"]//li[@class="a-last"]', navegador)
        next_page.click()
    except NoSuchElementException:
        print('Não foi possível encontrar mais avaliações.')
        break
    except Exception as e:
        print(e)
        #print('Não foi possível encontrar mais avaliações.')
        break

#pickle.dump(all_reviews_text, open('all_reviews_text.pkl', 'wb'))

Não foi possível encontrar mais avaliações.


In [41]:
pickle.dump(all_reviews_text, open('all_reviews_text.pkl', 'wb'))

In [37]:
pickle.dump(all_stars_text, open('all_stars_text.pkl', 'wb'))

In [19]:
pickle.load(open('all_reviews_text1.pkl', 'rb'))

[['Afterhourshop\nNot bad AT ALL. Better for me than the $2K X-Rite software I spent.\nReviewed in the United States on October 26, 2020\nVerified Purchase\nOk, so I have the $2K i1 Pro2 and I got this as I wanted to compare them.\nI am NOT going into deep details, but I can say that this hands down does a great job of making your prints MUCH better with fine tuning the colors and tones. I used the Expert patchs of about 750 and with Gray tones, and I used the Click method of EACH patch of color. This took some time and patients, BUT, it is the right way to go. The other method is LAME, and dont EVEN think about printing on a large sheet with more patches. WASTE of time. Print on a 8.5x11 or something so that cheap plastic paper guide that is included can actually help. This is where the i1 X-Rite is far superior, but SURELY not worth the $ extra for personal use.\n\nSo do the readings patch at a time, and you will be sure you got a good reading. My BW prints using color inks is not nu

In [20]:
_sum = 0
for review_list in all_reviews_text:
    _sum += len(review_list)
_sum

28